In [99]:
from faker import Faker
import random
import enum

In [100]:
faker = Faker("ru-RU")

In [200]:
class Singleton:
    def __new__(cls, *args, **kwargs):
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
            
        return cls.instance

    
class OrderStates(enum.Enum):
    accepted = "ORDER_ACCEPTED"
    sent = "ORDER_SENT"
    received = "ORDER_RECEIVED"
    

class Observer:
    def __init__(self, name):
        self.name = name
        self.state = None
        
    def update_state(self, new_state: OrderStates):
        self.state = new_state
        print("[Observer] Got new state:", self.state)

In [232]:
class DeliveryStrategy:
    def __init__(self, strategy):
        valid_strategies = [PostDeliveryStrategy, CourierDeliveryStrategy, MyselfDeliveryStrategy]

        if strategy not in valid_strategies:
            raise Exception('Выбрана несуществующая стратегия доставки.')
        
        self.strategy = strategy
        
    
    def start(self, address: str):
        self.strategy.start(self, address)


class PostDeliveryStrategy(DeliveryStrategy):
    def start(self, address: str):
        print(f"Доставка на адрес {address}: Ближайшее почтовое отделение")


class CourierDeliveryStrategy(DeliveryStrategy):
    def start(self, address: str):
        print(f"Доставка на адрес {address}: Курьер")


class MyselfDeliveryStrategy(DeliveryStrategy):
    def start(self, address: str):
        print("Самовывоз")

In [233]:
class Admin:
    def __init__(self, name: str, surname: str, phone: str):
        self.name = name
        self.surname = surname
        self.phone = phone
        
    def __repr__(self):
        return self.name
    
    def __str__(self):
        return self.name
    

class Order:
    def __init__(self, user: str, address: str, phone: str, product: any, delivery: DeliveryStrategy):
        self.id = random.randint(1, 1000000)
        self.user = user
        self.delivery = delivery
        self.phone = phone
        self.product = product
        self.state = OrderStates.accepted
        self.closed = False
        self.observers = [Observer("test")]
        
    def change_state(self, state: OrderStates):
        if self.closed:
            return
        self.state = state
        self.__notificate(state)
        
    def close(self):
        self.closed = True
        
    def __notificate(self, state: OrderStates):
        if self.closed:
            return
        
        for observer in self.observers:
            if hasattr(observer, 'update_state'):
                observer.update_state(state)
                
    def __str__(self):
        return f"{str(self.id)} ({self.state})"
    
    def __repr__(self):
        return f"{str(self.id)} ({self.state})"
        
        

class Shop(Singleton): 
    def __init__(self, name: str, admin: Admin):
        self.name = name
        self.admin = admin
        self.orders = []
        
    def add_order(self, order: Order):
        if order.closed:
            return

        self.orders.append(order)
        
    def change_order_state(self, order_id: int, state: OrderStates):
        for order in self.orders:
            if order.id == order_id:
                order.change_state(state)

In [234]:
shop = Shop("AllWhatYouNeed", Admin("John", "Doyle", "7 999 545 99 78"))

In [235]:
class Book:
    def __init__(self, name: str, author: str):
        self.name = name
        self.author = author
        
    def __str__(self):
        return f'{self.name} ({self.author})'
    
    def __repr__(self):
        return f'{self.name} ({self.author})'
        
    
class Electronic:
    def __init__(self, device: str, name: str, model: str):
        self.device = device
        self.name = name
        self.model = model

    def __str__(self):
        return f'{self.name} ({self.model})'

    def __repr__(self):
        return f'{self.name} ({self.model})'
    
    

class Clothes:
    def __init__(self, name: str, brand: str):
        self.name = name
        self.brand = brand

    def __str__(self):
        return f'{self.brand} - {self.name}'

    def __repr__(self):
        return f'{self.brand} - {self.name}'
    

class Product:
    def __init__(self, product_type: str, args: dict):
        self.product_type = product_type
        self.args = args
        self.products_dict = {
            'book': Book,
            'electronic': Electronic,
            'clothes': Clothes
        }
        
    def get_product(self):
        if self.product_type.lower() not in self.products_dict:
            raise Exception("Ваш тип продукта является неизвестным.")
        
        for key, cls in self.products_dict.items():
            if key == self.product_type:
                try:
                    return cls(**self.args)
                except Exception as e:
                    print("Ошибка в инициализации объекта продукта:\n", e)
                    return 

In [236]:
product = Product("clothes", {"name": "Сумка", "brand": "Gucci"}).get_product()
product

Gucci - Сумка

In [242]:
shop.add_order(Order("John", "Main St.", "7 929 559 95 96", product, DeliveryStrategy(CourierDeliveryStrategy)))

In [243]:
shop.orders

[437330 (OrderStates.sent), 79623 (OrderStates.accepted)]

In [244]:
shop.change_order_state(437330, OrderStates.sent)

[Observer] Got new state: OrderStates.sent


In [245]:
shop.orders[0].delivery.start("Мой адрес")

Доставка на адрес Мой адрес: Курьер
